In [1]:
%matplotlib ipympl
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pkg_resources
import pykonal

In [2]:
fname = pkg_resources.resource_filename(
    'pykonal',
    os.path.join('pykonal', 'data', 'marmousi_2d.npz')
)

In [3]:
solver = pykonal.EikonalSolver()
with np.load(fname) as infile:
    solver.vgrid.min_coords     = infile['min_coords']
    solver.vgrid.node_intervals = infile['node_intervals']
    solver.vgrid.npts           = infile['vv'].shape
    solver.vv                   = infile['vv']

solver.pgrid.min_coords     = solver.vgrid.min_coords
solver.pgrid.node_intervals = solver.vgrid.node_intervals
solver.pgrid.npts           = solver.vgrid.npts
solver.add_source((1,1,0))
solver.solve()
solver.determinant_errors, solver.denominator_errors

In [ ]:
vxy = np.moveaxis(
    np.stack(
        np.meshgrid(
            np.linspace(
                solver.vgrid.min_coords[0],
                solver.vgrid.max_coords[0]+solver.vgrid.node_intervals[0],
                solver.vgrid.npts[0]+1
            ),
            np.linspace(
                solver.vgrid.min_coords[1],
                solver.vgrid.max_coords[1]+solver.vgrid.node_intervals[1],
                solver.vgrid.npts[1]+1
            ),
            indexing='ij'
        ),
    ),
    0, -1
) - solver.vgrid.node_intervals[:2] / 2

pxy = np.moveaxis(
    np.stack(
        np.meshgrid(
            np.linspace(
                solver.pgrid.min_coords[0],
                solver.pgrid.max_coords[0]+solver.pgrid.node_intervals[0],
                solver.pgrid.npts[0]+1
            ),
            np.linspace(
                solver.pgrid.min_coords[1],
                solver.pgrid.max_coords[1]+solver.pgrid.node_intervals[1],
                solver.pgrid.npts[1]+1
            ),
            indexing='ij'
        ),
    ),
    0, -1
) - solver.pgrid.node_intervals[:2] / 2

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(2, 1, 1, aspect=1)
ax.pcolormesh(
    vxy[:,:,0],
    vxy[:,:,1],
    solver.vv[:,:,0],
    cmap=plt.get_cmap('jet_r')
)
ax.invert_yaxis()
ax = fig.add_subplot(2, 1, 2, aspect=1)
ax.pcolormesh(pxy[:,:,0], pxy[:,:,1], solver.uu[:,:,0], cmap=plt.get_cmap('jet_r'))
ax.invert_yaxis()